This example notebook was originally developed by Lynne Jones (@rhiannonlynne) and subsequently updated.

**This notebook is currently under revision and not working properly. Please refer to the examples provided in the documentation for now.**

In [1]:
import os
import numpy as np
import pyoorb as oo

### Initialize Oorb

In [2]:
# Initialize oorb
oo.pyoorb.oorb_init()

0

In [3]:
timeScales = {'UTC': 1, 'UT1': 2, 'TT': 3, 'TAI': 4}
elemType = {'CART': 1, 'COM': 2, 'KEP': 3, 'DEL': 4, 'EQX': 5}

### Read in some orbits.

In [4]:
# Set up some orbits
# orb is id, 6 elements, epoch_mjd, H, G, element type index
# keplerian appears to be element type index 3
# orbits = numpy.array([0.,1.,2.,3.,4.,5.,6.,5373.,1.,1.,3.])
o = np.loadtxt('test.des', dtype=([('objid', np.str_, 20), ('format', np.str_, 3),
                                   ('q', float), ('e', float), ('inc', float), ('Omega', float),
                                   ('argperi', float), ('tperi', float), ('H', float), ('epoch', float),
                                   ('index', int), ('npar', int), ('moid', float), ('compcode', np.str_, 10)]),
               skiprows=1)

orbits = np.zeros([len(o), 12], dtype=np.double, order='F')
orbits[:, 0] = np.arange(0, len(o), 1, dtype=float)
orbits[:, 1] = o['q']
orbits[:, 2] = o['e']
orbits[:, 3] = o['inc']
orbits[:, 4] = o['Omega']
orbits[:, 5] = o['argperi']
orbits[:, 6] = o['tperi']
orbits[:, 7] = np.zeros(len(o), float) + elemType['COM']
orbits[:, 8] = o['epoch']
orbits[:, 9] = np.zeros(len(o), float) + timeScales['TT']
orbits[:, 10] = o['H']
orbits[:, 11] = np.zeros(len(o)) + 0.15
for i in range(3, 6):
    orbits[:, i] = np.radians(orbits[:, i])

In [5]:
orbits

array([[0.00000000e+00, 2.72680464e+00, 1.43389847e-01, 1.11295242e-01,
        1.69610904e+00, 5.11783260e+00, 5.37715237e+04, 2.00000000e+00,
        5.44660000e+04, 3.00000000e+00, 1.75820000e+01, 1.50000000e-01],
       [1.00000000e+00, 2.64978782e+00, 5.60540098e-02, 1.36881230e-01,
        2.99464483e+00, 2.50204097e+00, 5.49679725e+04, 2.00000000e+00,
        5.44660000e+04, 3.00000000e+00, 2.05140000e+01, 1.50000000e-01],
       [2.00000000e+00, 2.82327567e+00, 8.02216088e-02, 2.03150687e-01,
        1.70781814e+00, 5.79273522e+00, 5.36066833e+04, 2.00000000e+00,
        5.44660000e+04, 3.00000000e+00, 1.83040000e+01, 1.50000000e-01]])

### Generate ephemerides

This does n-body propagation to the series of times specified.

In [6]:
offset = 30.
times = [orbits[0][8], orbits[0][8] + offset]
timescale = [timeScales['UTC']] * len(times)
ephem_dates = np.array(list(zip(times, timescale)), dtype=np.double, order='F')
ephs, err = oo.pyoorb.oorb_ephemeris_full(in_orbits=orbits,
                                          in_obscode='I11',
                                          in_date_ephems=ephem_dates,
                                          in_dynmodel='N')
if err != 0:
    print(err)

In [7]:
i = 0
start = ephs[i][0]
end = ephs[i][1]
print("@ %f" % (ephem_dates[0][0]), start)
print("@ %f" % (ephem_dates[1][0]), end)

@ 54466.000000 [ 5.44660000e+04  1.82578901e+02  5.88067581e+00  1.04457023e-01
 -3.41723035e-03  1.62736031e+01  9.97366497e+01  3.45839825e+00
  3.15354150e+00  2.36498516e+01  9.18737187e+01  1.80017867e+02
  6.41945377e+00  8.02010329e+01  6.42190219e+00  1.63659807e+02
  5.85080877e+00  6.38429728e+01  5.85325719e+00 -5.51502279e+01
 -3.21957926e+00 -4.77527098e+01  4.36034683e-01  1.96617818e+01
 -3.30141908e+00  9.67916978e-01  3.52543923e-01 -3.39966388e-03
 -8.14672055e-03  4.90750232e-04 -1.67650276e-01  9.68894212e-01
 -4.20190092e-05  1.33382199e+02]
@ 54496.000000 [ 5.44960000e+04  1.83595895e+02  6.69572325e+00 -3.43432561e-02
  5.77223431e-02  1.26202682e+01  1.29325279e+02  3.48774043e+00
  2.77919798e+00  2.32687273e+01  3.29248482e+02  1.80622608e+02
  7.57092179e+00  5.02602926e+01  7.57334218e+00  1.68007840e+02
  6.02574931e+00  3.76455244e+01  6.02816970e+00 -3.33027149e+01
 -4.29243384e+00 -3.65275117e+01  4.25902627e-01  4.96610420e+01
 -3.39277428e+00  7.206712

### Transform orbital elements

In [8]:
type2 = 'CART'
print("Transforming from COM to %s" % (type2))
newElems, err = oo.pyoorb.oorb_element_transformation(in_orbits=orbits, in_element_type=elemType[type2])
if err != 0:
    print("error", err)
print("Input and output orbital elements (COM / CART)")
print("in ", orbits[i])
print("out", newElems[i])

Transforming from COM to CART
Input and output orbital elements (COM / CART)
in  [0.00000000e+00 2.72680464e+00 1.43389847e-01 1.11295242e-01
 1.69610904e+00 5.11783260e+00 5.37715237e+04 2.00000000e+00
 5.44660000e+04 3.00000000e+00 1.75820000e+01 1.50000000e-01]
out [ 0.00000000e+00 -3.30147843e+00  9.67774744e-01  3.52552490e-01
 -3.39925149e-03 -8.14684133e-03  4.90706192e-04  1.00000000e+00
  5.44660000e+04  3.00000000e+00  1.75820000e+01  1.50000000e-01]


In [9]:
print("Transforming back from %s to COM" % (type2))
newElems2, err = oo.pyoorb.oorb_element_transformation(in_orbits=newElems, in_element_type=elemType['COM'])
if err != 0:
    print("error", err)
print("Input and output orbital elements (CART / COM)")
print("in ", newElems[i])
print("out", newElems2[i])

Transforming back from CART to COM
Input and output orbital elements (CART / COM)
in  [ 0.00000000e+00 -3.30147843e+00  9.67774744e-01  3.52552490e-01
 -3.39925149e-03 -8.14684133e-03  4.90706192e-04  1.00000000e+00
  5.44660000e+04  3.00000000e+00  1.75820000e+01  1.50000000e-01]
out [0.00000000e+00 2.72680464e+00 1.43389847e-01 1.11295242e-01
 1.69610904e+00 5.11783260e+00 5.37715237e+04 2.00000000e+00
 5.44660000e+04 3.00000000e+00 1.75820000e+01 1.50000000e-01]


In [13]:
print('Differences in the initial/final COM orb elements')
diffs = newElems2 - orbits
print(diffs)
print(newElems2[1,3])
print(orbits[1,3])
print(diffs[1,3])

Differences in the initial/final COM orb elements
[[ 0.00000000e+00 -1.73194792e-14  3.21964677e-15 -7.63278329e-16
   6.66133815e-16 -1.77635684e-15  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00 -2.08721929e-14  3.63598041e-15  9.99200722e-16
  -5.15143483e-14  9.90318938e-14  1.45519152e-11  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00 -2.53130850e-14  3.05311332e-15  1.66533454e-16
  -2.22044605e-16  7.63833441e-14  2.91038305e-11  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]]
0.13688122988932078
0.13688122988931978
9.992007221626409e-16


There can be larger differences in the tPeri, as this is degenerate (previous or next orbit?). 

### Propagate orbits

You can do both n-body and 2-body propagation (swap `in_dynmodel` from `'N'` to `'2'`). 
At the moment, oorb\_propagation\* crashes if you use anything other than CART (cartesian) elements. So transform first.

In [14]:
print("Propagating orbits by %d days" % (offset))
epoch_orig = orbits[i][8]
epoch_new = epoch_orig + offset
newEpoch = np.array([epoch_new, timeScales['TT']], dtype='double', order='F')
# Note that currently you have to use CART format elements.
elems = newElems
newOorbElems, err = oo.pyoorb.oorb_propagation(in_orbits=elems, in_epoch=newEpoch, in_dynmodel='N')
if err != 0:
    print("error", err)
print("Input and output orbital elements (0 and %d days)" % (offset))
print("in ", orbits[i])
print("out", newOorbElems[i])

Propagating orbits by 30 days
Input and output orbital elements (0 and 30 days)
in  [0.00000000e+00 2.72680464e+00 1.43389847e-01 1.11295242e-01
 1.69610904e+00 5.11783260e+00 5.37715237e+04 2.00000000e+00
 5.44660000e+04 3.00000000e+00 1.75820000e+01 1.50000000e-01]
out [ 0.00000000e+00 -3.39281543e+00  7.20543871e-01  3.66133292e-01
 -2.68965290e-03 -8.32572656e-03  4.14522375e-04  1.00000000e+00
  5.44960000e+04  3.00000000e+00  1.75820000e+01  1.50000000e-01]


In [15]:
print("Transforming new orbital elements to COM")
newElems3, err = oo.pyoorb.oorb_element_transformation(in_orbits=newOorbElems, in_element_type=elemType['COM'])
if err != 0:
    print("error", err)
print("Input and output orbital elements (CART/COM)")
print("in ", orbits[i])
print("out", newElems3[i])
print('epoch', epoch_new, newElems3[i][8])

Transforming new orbital elements to COM
Input and output orbital elements (CART/COM)
in  [0.00000000e+00 2.72680464e+00 1.43389847e-01 1.11295242e-01
 1.69610904e+00 5.11783260e+00 5.37715237e+04 2.00000000e+00
 5.44660000e+04 3.00000000e+00 1.75820000e+01 1.50000000e-01]
out [0.00000000e+00 2.72662741e+00 1.43413828e-01 1.11295174e-01
 1.69610744e+00 5.11761118e+00 5.37714875e+04 2.00000000e+00
 5.44960000e+04 3.00000000e+00 1.75820000e+01 1.50000000e-01]
epoch 54496.0 54496.0


In [17]:
# Generate ephemerides with these new elements, to compare.
ephs2, err = oo.pyoorb.oorb_ephemeris_full(in_orbits=newElems3, in_obscode='I11',
                                     in_date_ephems=ephem_dates, in_dynmodel='N')
for i in range(len(newElems3)):
    print('Object %d' % i)
    print(' Orbit: ', orbits[i] )
    diffs = ephs[i] - ephs2[i]
    #print("Difference in ephemerides from ephemeris generation vs propagation + ephemeris generation")
    #print(diffs)
    print("Just difference in mas in RA and then Dec")
    deltaRA = diffs[:,1] * 3600. * 1000.
    deltaDec = diffs[:,2] * 3600. * 1000.
    print(' ', deltaRA, deltaDec)

Object 0
 Orbit:  [0.00000000e+00 2.72680464e+00 1.43389847e-01 1.11295242e-01
 1.69610904e+00 5.11783260e+00 5.37715237e+04 2.00000000e+00
 5.44660000e+04 3.00000000e+00 1.75820000e+01 1.50000000e-01]
Just difference in mas in RA and then Dec
  [2.69096745e-05 3.10024006e-05] [-1.12262200e-05 -1.31830546e-05]
Object 1
 Orbit:  [1.00000000e+00 2.64978782e+00 5.60540098e-02 1.36881230e-01
 2.99464483e+00 2.50204097e+00 5.49679725e+04 2.00000000e+00
 5.44660000e+04 3.00000000e+00 2.05140000e+01 1.50000000e-01]
Just difference in mas in RA and then Dec
  [2.62957656e-05 3.01838554e-05] [-4.87929697e-06 -4.71942485e-06]
Object 2
 Orbit:  [2.00000000e+00 2.82327567e+00 8.02216088e-02 2.03150687e-01
 1.70781814e+00 5.79273522e+00 5.36066833e+04 2.00000000e+00
 5.44660000e+04 3.00000000e+00 1.83040000e+01 1.50000000e-01]
Just difference in mas in RA and then Dec
  [-2.55795385e-06 -2.96722646e-06] [8.76099193e-07 8.12150347e-07]


Note that the difference can change depending on the length of time in 'offset'. Needs a bit further investigation, but I suspect this is due to the timescale of integration hard-coded into the different routines (?). 